workflow simplifier notebook

In [7]:
# Import necessary packages
import os
from glob import glob
import matplotlib.pyplot as plt
from matplotlib import patches as mpatches, colors
import numpy as np
from shapely.geometry import box
import geopandas as gpd
import rasterio as rio
from rasterio.plot import plotting_extent
from rasterio.mask import mask
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep
import earthpy.mask as em

# Imports for box function?


# Get data and set working directory
data = et.data.get_data('ndvi-automation')
os.chdir(os.path.join(et.io.HOME, 'earth-analytics'))

In [8]:
# Import SJER site boundary (.shp)
sjer_base_path = os.path.join('data', 'ndvi-automation', 'sites', 'SJER')
sjer_site_boundary_path = os.path.join(
    sjer_base_path, 'vector', 'SJER-crop.shp')
sjer_site_boundary = gpd.read_file(sjer_site_boundary_path, masked=True)

# Import HARV site boundary (.shp)
harv_base_path = os.path.join('data', 'ndvi-automation', 'sites', 'HARV')
harv_site_boundary_path = os.path.join(
    harv_base_path, 'vector', 'HARV-crop.shp')
harv_site_boundary = gpd.read_file(harv_site_boundary_path, masked=True)

In [9]:
# Make a list of directories via glob
path = os.path.join('data','ndvi-automation','*','*','landsat-crop','*')
all_dirs = glob(path)

In [10]:
# qa = glob(os.path.join(all_dirs[0] + '/*pixel_qa*'))
# qa
all_bands = glob(os.path.join(all_dirs[0])+'/*band*.tif')
all_bands.sort()


In [11]:
type(all_bands)

list

In [12]:
with rio.open(all_dirs[0]) as src:
    test = src.read()

RasterioIOError: 'data/ndvi-automation/sites/SJER/landsat-crop/LC080420342017090401T1-SC20181023162756' not recognized as a supported file format.

Now that we have a list of all the directories the next step is to:
* glob all the files in each directory, extract the qa file, stack the bands to an output tif, import the output tif via context manager, calculate ndvi

In [ ]:
# Automation baby!!!
all_ndvi = []

for directory in all_dirs:
    # Extract the date of the directory 
    date = os.path.basename(os.path.normpath(directory))
    date = date[10:18]
    all_ndvi.append(date)
    
    # Extract qa and open via context manager
    qa_glob = glob(os.path.join(directory + '/*pixel_qa*.tif'))
    with rio.open(qa_glob[0]) as qa_src:
        qa = qa_src.read(1)
    
    # Exctract all_bands(sorted) via glob from each directory
    all_bands_glob = glob(os.path.join(directory + '/*band*.tif'))
    all_bands_glob.sort()
    
    # Run stack function
    output_path = os.path.join('data','ndvi-automation','outputs','file.tif')
    land_stack, land_meta = es.stack(all_bands_glob, output_path)
    
     
    # Open stack ouput .tif and read all_bands via context manager
    with rio.open(output_path) as bands_src:
        all_bands = bands_src.read() # Missing cloud management (come back to this)
                               # Also missing: clipping to the site boundary (come back to this)
    
#     # Mask for cloud cover
#     high_cloud_confidence = em.pixel_flags["pixel_qa"]["L8"]["High Cloud Confidence"]
#     all_bands_free = em.mask_pixels(arr=all_bands, mask_arr=qa, vals=high_cloud_confidence)
    
    
    # Calculate NDVI
    ndvi = es.normalized_diff(all_bands[4], all_bands[3]) 
    
    # Save ndvi's to this list
    all_ndvi.append(ndvi)
    
    
len(all_ndvi) 